In [1]:
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('MLlib Linear Regression').getOrCreate()

In [3]:
spark

In [13]:
training=spark.read.csv('test4.csv', header=True, inferSchema=True)

In [14]:
training=df_pyspark.na.drop()

In [15]:
training.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [16]:
training.columns

['Name', 'age', 'Experience', 'Salary']

In [17]:
# Vector Assembler
# [dependent_feature_1, dependent_feature_2] -----> new feature -----> independent feature
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=['age','Experience'], outputCol="Independent Feature")

In [18]:
output=featureassembler.transform(training)

In [19]:
output.show()

+---------+---+----------+------+-------------------+
|     Name|age|Experience|Salary|Independent Feature|
+---------+---+----------+------+-------------------+
|    Krish| 31|        10| 30000|        [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|         [30.0,8.0]|
|    Sunny| 29|         4| 20000|         [29.0,4.0]|
|     Paul| 24|         3| 20000|         [24.0,3.0]|
|   Harsha| 21|         1| 15000|         [21.0,1.0]|
|  Shubham| 23|         2| 18000|         [23.0,2.0]|
+---------+---+----------+------+-------------------+



In [20]:
# 'Independent Feature' is used as input for the model
finalised_data=output.select(['Independent Feature', 'Salary'])
finalised_data.show()

+-------------------+------+
|Independent Feature|Salary|
+-------------------+------+
|        [31.0,10.0]| 30000|
|         [30.0,8.0]| 25000|
|         [29.0,4.0]| 20000|
|         [24.0,3.0]| 20000|
|         [21.0,1.0]| 15000|
|         [23.0,2.0]| 18000|
+-------------------+------+



In [33]:
from pyspark.ml.regression import LinearRegression

train_data, test_data=finalised_data.randomSplit([0.70,0.30])
regressor=LinearRegression(featuresCol='Independent Feature', labelCol='Salary')
regressor=regressor.fit(train_data)

In [34]:
regressor.coefficients

DenseVector([-250.0, 1750.0])

In [35]:
regressor.intercept

20250.0000000005

In [39]:
result=regressor.evaluate(test_data)
result.predictions.show()

+-------------------+------+------------------+
|Independent Feature|Salary|        prediction|
+-------------------+------+------------------+
|         [21.0,1.0]| 15000| 16750.00000000007|
|         [24.0,3.0]| 20000|19500.000000000036|
|         [30.0,8.0]| 25000|26749.999999999993|
+-------------------+------+------------------+

